In [24]:
cd /home/nhan-softzone/law_search

/home/nhan-softzone/law_search


/home/nhan-softzone/miniconda3/envs/law_agent/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import random
import numpy as np
import pandas as pd
from datasets import load_dataset
from modules.eval.hybrid_engine import HybridEvaluationEngine

# Configuration
bm25_weight = 0.5
dense_weight = 0.5
rrf_k = 60.0
batch_size = 32
dimension = 768

models = {
    "bkai": {
        "model_name": "bkai-foundation-models/vietnamese-bi-encoder",
        "word_segment": True
    },
    "halong": {
        "model_name": "hiieu/halong_embedding",
        "word_segment": False
    }
}

dataset_name_base = "another-symato/VMTEB-Zalo-legel-retrieval"
num_corpus_addition = 0

eval_results = {}

for model_key, model_config in models.items():
    dataset_name = dataset_name_base + "-wseg" if model_config["word_segment"] else dataset_name_base
    
    # Load datasets
    corpus = load_dataset(dataset_name, "corpus", split='train')
    queries = load_dataset(dataset_name, "queries", split='train')
    relevant_docs_data = load_dataset(dataset_name, "data_ir", split="test")
    
    # Filter corpus and queries based on relevant docs
    required_corpus_ids = list(map(str, relevant_docs_data["corpus_id"]))
    if num_corpus_addition:
        required_corpus_ids += random.sample(corpus["id"], k=num_corpus_addition)
    corpus = corpus.filter(lambda x: x["id"] in required_corpus_ids)
    
    required_queries_ids = list(map(str, relevant_docs_data["query_id"]))
    queries = queries.filter(lambda x: x["query_id"] in required_queries_ids)
    
    # Convert to dictionary format
    corpus_dict = dict(zip(corpus["id"], corpus["text"]))
    queries_dict = dict(zip(queries["query_id"], queries["question"]))
    relevant_docs = {}
    for qid, corpus_id in zip(relevant_docs_data["query_id"], relevant_docs_data["corpus_id"]):
        qid = str(qid)
        corpus_id = str(corpus_id)
        if qid not in relevant_docs:
            relevant_docs[qid] = set()
        relevant_docs[qid].add(corpus_id)
    
    # Initialize evaluation engine
    engine = HybridEvaluationEngine(
        model_name=model_config["model_name"],
        bm25_weight=bm25_weight,
        dense_weight=dense_weight,
        rrf_k=rrf_k,
        dimension=dimension
    )
    
    # Run evaluation
    eval_results[model_key] = engine.evaluate(queries, corpus, relevant_docs)

# Create score board
df_score_board = pd.DataFrame(eval_results)

Indexing documents...


  0%|          | 0/640 [00:00<?, ?it/s]                           

Evaluating queries...


100%|██████████| 640/640 [00:53<00:00, 11.99it/s]


Indexing documents...


  0%|          | 0/640 [00:00<?, ?it/s]                           

Evaluating queries...


100%|██████████| 640/640 [00:53<00:00, 12.01it/s]

                  bkai    halong
ndcg@3        0.927286  0.873138
ndcg@5        0.930580  0.892030
ndcg@10       0.929662  0.898843
mrr@3         0.910677  0.855729
mrr@5         0.912682  0.867318
mrr@10        0.911202  0.871318
recall@3      0.975000  0.923438
recall@5      0.982812  0.965625
recall@10     0.984375  0.982812
precision@3   0.325000  0.307812
precision@5   0.196562  0.193125
precision@10  0.098437  0.098281
f1@3          0.487500  0.461719
f1@5          0.327604  0.321875
f1@10         0.178977  0.178693


In [27]:
df_score_board.to_json('output/scores.json')